**Einlesen der Daten**

In [ ]:
# TODO: pandas importieren

df = pd.read_csv("trainingsdaten_ohne_vibration.csv", encoding = "ISO-8859-1")
df_test = pd.read_csv("testdaten_mit_vibration.csv", encoding = "ISO-8859-1")

**Trainingsdaten anschauen**

In [ ]:
df.head()

**Testdaten anschauen**

In [ ]:
df_test.head()

***Time* löschen**

In [ ]:
del df["Time"]
del df_test["Time"]

**Deskriptive Statistik (Trainingsdaten)**

In [ ]:
df.describe()

**Deskriptive Statistik (Testdaten)**

In [ ]:
# TODO: Deskriptive Statistik ausgeben analog zu oben (Trainingsdaten)

**Daten visualisieren**

In [ ]:
import matplotlib.pyplot as plt

df.plot(subplots=True, layout=(3,2), legend=False)
plt.show()

In [ ]:
df_test.plot(subplots=True, layout=(3,2), legend=False)
plt.show()

**Datenvorverarbeitung**

In [ ]:
# 2 Spalten enthalten keine Information -> löschen
df = df.drop(['NumberOverload', 'NumberUnderloads'], 1)
df_test = df_test.drop(['NumberOverload', 'NumberUnderloads'], 1)

**Test- und Trainingsdaten festlegen**

In [ ]:
X_train = df
X_test = df_test[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].copy()
y_test = df_test[['VibrationMotorOn']].copy()

**Modell trainieren**

In [ ]:
# TODO: IsolationForest aus sklearn importieren

# Modell erstellen
clf = IsolationForest(random_state=42)

# TODO: Modell an Trainingsdaten anpassen ("fit")

pred = pd.DataFrame(columns = ["Anomaly"])
# Vorhersage des Modells
for k in range(len(X_test)):
    temp = clf.predict([[X_test["AmplitudeBandWidth"][k], X_test["AmplitudeMean"][k], X_test["StabilizationTime"][k]]])[0]
    pred = pred.append({"Anomaly": temp}, ignore_index = True)

# Vorhersage des Modells auf 1/0 recodieren
pred['Anomaly'] = pred['Anomaly'].map( {1: 0, -1: 1} )

**Metriken ausgeben**

In [ ]:
from sklearn import metrics

print ("Accuracy: ")
# TODO: Accuracy Score ausgeben
print ("Classification Report:")
print (metrics.classification_report(y_test, pred))
print ("Confusion Matrix:")
print (metrics.confusion_matrix(y_test, pred))  

**Daten zusammenführen für Visualisierung**

In [ ]:
complete_data = X_test.copy()
complete_data['VibrationMotorOn'] = y_test['VibrationMotorOn']
complete_data['Anomaly'] = pred.values

**Farben für Visualisierung einfügen**

In [ ]:
complete_data["color"] = 0
for idx,row in complete_data.iterrows():
    label_true = complete_data.loc[idx,"VibrationMotorOn"]
    label_pred = complete_data.loc[idx,"Anomaly"]
    if  label_true == True:
        if  label_pred == True:
            complete_data.loc[idx,"color"] = "blue"
        else:
            complete_data.loc[idx,"color"] = "green"
    else:
        if label_pred == False:
            complete_data.loc[idx,"color"] = "red"            
        else:
            complete_data.loc[idx,"color"] = "yellow"

**Visualisierung der Konfusionsmatrix**

In [ ]:
fig, ax = plt.subplots()
a = complete_data.copy()

ax.scatter(a.loc[a.color == 'blue'].index,a.loc[a.color == 'blue',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='blue', label='True positive')
ax.scatter(a.loc[a.color == 'green'].index,a.loc[a.color == 'green',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='green', label='False negative')
ax.scatter(a.loc[a.color == 'red'].index,a.loc[a.color == 'red',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='red', label='True negative')
# TODO: letzte Zeile einfügen

plt.legend()
plt.show()